In [116]:
import numpy as np
import sys
from ipycanvas import hold_canvas, Canvas

sys.path.append('../src')
from models import Vicsek
from util import RunTime

In [109]:
CANVAS_SIZE = 500
FARM_COLOR = '#1b1f22'
ANT_COLOR = 'red'
ANT_SIZE = 3.

def draw(x, canvas, canvas_size=500, farm_color='#1b1f22', ant_color='red', ant_size=3.):
    ants = np.unique(x[:, 0]).astype(np.int32)

    with hold_canvas():
        canvas.clear()
        canvas.fill_style = farm_color
        canvas.fill_rect(0, 0, width=canvas_size, height=canvas_size)

        for ant in ants:
            canvas.fill_style = ['red', 'blue', 'green', 'yellow'][ant - 1]
            canvas.fill_circles(*x[x[:, 0] == ant, 1:].T, ant_size)

def update():
    global model, canvas, x, v
    
    x[:, 1:], v = model(x[:, 1:], v)
    draw(x, canvas, CANVAS_SIZE, FARM_COLOR, ANT_COLOR, ANT_SIZE)

def toggle(*_):
    global runtime

    if runtime.running:
        runtime.stop()
    else:
        runtime.start()

def start(function, interval=.02):
    global runtime

    try:
        runtime.stop()
    except:
        pass

    runtime = RunTime(function, interval)

In [110]:
class Vicsek2:
    def __init__(self, proximity=20., velocity=3., update_rate=1., variance=.3, canvas_size=500):
        self.proximity = proximity
        self.velocity = velocity
        self.update_rate = update_rate
        self.variance = variance
        self.canvas_size = canvas_size

    def __call__(self, x, v):
        ants = np.unique(x[:, 0])

        for ant in ants:
            friends, enemies = x[:, 0] == ant, x[:, 0] != ant
            neighborhood = (np.sqrt(np.square(x[:, 1:][:, None] - x[:, 1:][None, :])).sum(-1) < self.proximity).astype(np.int32)
            neighborhood[:, enemies] = -1*neighborhood[:, enemies]
            uncertainty = np.random.normal(0, self.variance, size=v[friends].shape)
            v[friends] = (neighborhood[friends]@v)/np.abs(neighborhood[friends]).sum(-1)[:, None] + uncertainty
            alignment = np.hstack([np.cos(v[friends]), np.sin(v[friends])])
            x[friends, 1:] = (x[friends, 1:] + self.velocity*self.update_rate*alignment)%(self.canvas_size)

        return x, v

In [119]:
N_ANTS = 500
PROXIMITY = 20.
VELOCITY = 2.
UPDATE_RATE = 1.
VARIANCE = .2

x1 = np.hstack([1*np.ones((N_ANTS//4, 1)), np.random.rand(N_ANTS//4, 2)*CANVAS_SIZE])
x2 = np.hstack([2*np.ones((N_ANTS//4, 1)), np.random.rand(N_ANTS//4, 2)*CANVAS_SIZE])
x3 = np.hstack([3*np.ones((N_ANTS//4, 1)), np.random.rand(N_ANTS//4, 2)*CANVAS_SIZE])
x4 = np.hstack([4*np.ones((N_ANTS//4, 1)), np.random.rand(N_ANTS//4, 2)*CANVAS_SIZE])
x = np.vstack([x1, x2, x3, x4])
v = 2*np.pi*np.random.rand(N_ANTS, 1)
model = Vicsek(PROXIMITY, VELOCITY, UPDATE_RATE, VARIANCE, CANVAS_SIZE)
canvas = Canvas(width=CANVAS_SIZE, height=CANVAS_SIZE)
canvas.on_mouse_down(toggle)

display(canvas)
draw(x, canvas, CANVAS_SIZE, FARM_COLOR, ANT_COLOR, ANT_SIZE)
start(update)

Canvas(width=500)